# Preprocessing of TED CAN (contract award notices) CSV files

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
from pyspark.sql import HiveContext
import json
import time
import sys
from datetime import datetime
from pyspark.sql.types import *
import re

nPartitions=4
conf = (SparkConf()
         .setMaster("local["+str(nPartitions)+"]")
#         .setAppName(analysisName)
       )
sc.stop()
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
sqlContext.sql("SET spark.sql.parquet.binaryAsString=true")


DataFrame[key: string, value: string]

## Load country code file
Used to match country ISO codes to country full names

In [32]:
countryCodesRaw = [line.rstrip() for line in open('data/countrycodes.csv')]

In [33]:
countryCodes = {}
for i in range(len(countryCodesRaw)):
    code_country=countryCodesRaw[i].split(' - ')
    countryCodes[code_country[0]] = code_country[1]
countryCodes['']=''

## CSV row processor and schema
Convert CSV rows to nicer format and schema

In [57]:
def process(row):
    
    dispatch_date=datetime.strptime(row['DT_DISPATCH'], '%d-%b-%y').strftime('%Y-%m-%d')
    award_notice_id=row['ID_NOTICE_CAN'].strip()
    award_notice_id=award_notice_id[4:]+'-'+award_notice_id[0:4]
    award_notice_id_link="<a href='http://ted.europa.eu/udl?uri=TED:NOTICE:"+award_notice_id+":TEXT:EN:HTML' target='_blank'>"+award_notice_id+"</a>"
    
    contracting_authority_country=countryCodes[row['ISO_COUNTRY_CODE'].strip()]
    contracting_authority_name=row['CAE_NAME'].replace ("_", "-").strip()
    
    contractor_country=countryCodes[row['WIN_COUNTRY_CODE'].strip()]
    contractor_name=row['WIN_NAME'].replace ("_", "-").strip()
    
    contract_value_euros=None if row['AWARD_VALUE_EURO']=='' else int(float(row['AWARD_VALUE_EURO']))
    number_offers_received=None if row['NUMBER_OFFERS']=='' else int(row['NUMBER_OFFERS'])
    
    CPV_code=row['CPV'].strip()
    
    YEAR=None if row['YEAR']=='' else int(row['YEAR'])
    ID_TYPE=row['ID_TYPE'].strip()
    XSD_VERSION=row['XSD_VERSION'].strip()
    CANCELLED=row['CANCELLED'].strip()
    
    CAE_NATIONALID=row['CAE_NATIONALID'].strip()
    CAE_ADDRESS=row['CAE_ADDRESS'].strip()
    CAE_TOWN=row['CAE_TOWN'].strip()
    CAE_POSTAL_CODE=row['CAE_POSTAL_CODE'].strip()
    
    CAE_TYPE=row['CAE_TYPE'].strip()
    MAIN_ACTIVITY=row['MAIN_ACTIVITY'].strip()
    B_ON_BEHALF=row['B_ON_BEHALF'].strip()
    TYPE_OF_CONTRACT=row['TYPE_OF_CONTRACT'].strip()
    TAL_LOCATION_NUTS=row['TAL_LOCATION_NUTS'].strip()
    B_FRA_AGREEMENT=row['B_FRA_AGREEMENT'].strip()
    B_DYN_PURCH_SYST=row['B_DYN_PURCH_SYST'].strip()
    ADDITIONAL_CPVS=row['ADDITIONAL_CPVS'].strip()
    B_GPA=row['B_GPA'].strip()
    VALUE_EURO_FIN_1=None if row['VALUE_EURO_FIN_1']=='' else int(float(row['VALUE_EURO_FIN_1']))
    VALUE_EURO_FIN_2=None if row['VALUE_EURO_FIN_2']=='' else int(float(row['VALUE_EURO_FIN_2']))
    TOP_TYPE=row['TOP_TYPE'].strip()
    CRIT_CODE=row['CRIT_CODE'].strip()
    CRIT_CRITERIA=row['CRIT_CRITERIA'].strip()
    CRIT_WEIGHTS=row['CRIT_WEIGHTS'].strip()
    B_ELECTRONIC_AUCTION=row['B_ELECTRONIC_AUCTION'].strip()
    NUMBER_AWARDS=None if row['NUMBER_AWARDS']=='' else int(float(row['NUMBER_AWARDS']))
    
    WIN_ADDRESS=row['WIN_ADDRESS'].strip()
    WIN_TOWN=row['WIN_TOWN'].strip()
    WIN_POSTAL_CODE=row['WIN_POSTAL_CODE'].strip()
    
    CONTRACT_NUMBER=row['CONTRACT_NUMBER'].strip()
    LOT_NUMBER=row['LOT_NUMBER'].strip()
    TITLE=row['TITLE'].strip()
    NUMBER_OFFERS_ELECTR=None if row['NUMBER_OFFERS_ELECTR']=='' else int(float(row['NUMBER_OFFERS_ELECTR'])) 
    AWARD_EST_VALUE_EURO=None if row['AWARD_EST_VALUE_EURO']=='' else int(float(row['AWARD_EST_VALUE_EURO']))
    AWARD_VALUE_EURO=None if row['AWARD_VALUE_EURO']=='' else int(float(row['AWARD_VALUE_EURO']))
    VALUE_EURO_FIN_1_1=None if row['VALUE_EURO_FIN_1_1']=='' else int(float(row['VALUE_EURO_FIN_1_1']))
    B_SUBCONTRACTED=row['B_SUBCONTRACTED'].strip()
    B_EU_FUNDS=row['B_EU_FUNDS'].strip()
    DT_AWARD=None if row['DT_AWARD']=='' else datetime.strptime(row['DT_AWARD'].strip(), '%d-%b-%y').strftime('%Y-%m-%d')
    
    awardNoticeRow=[\
                    dispatch_date,\
                    award_notice_id_link,\
                    contracting_authority_country,\
                    contracting_authority_name,\
                    contractor_country,\
                    contractor_name,\
                    contract_value_euros,\
                    number_offers_received,\
                    CPV_code,\
                      
                    YEAR,\
                    ID_TYPE,\
                    XSD_VERSION,\
                    CANCELLED,\
                    
                    CAE_NATIONALID,\
                    CAE_ADDRESS,\
                    CAE_TOWN,\
                    CAE_POSTAL_CODE,\
                    
                    CAE_TYPE,\
                    MAIN_ACTIVITY,\
                    B_ON_BEHALF,\
                    TYPE_OF_CONTRACT,\
                    TAL_LOCATION_NUTS,\
                    B_FRA_AGREEMENT,\
                    B_DYN_PURCH_SYST,\
                    ADDITIONAL_CPVS,\
                    B_GPA,\
                    VALUE_EURO_FIN_1,\
                    VALUE_EURO_FIN_2,\
                    TOP_TYPE,\
                    CRIT_CODE,\
                    CRIT_CRITERIA,\
                    CRIT_WEIGHTS,\
                    B_ELECTRONIC_AUCTION,\
                    NUMBER_AWARDS,\
                    
                    WIN_ADDRESS,\
                    WIN_TOWN,\
                    WIN_POSTAL_CODE,\
                    
                    CONTRACT_NUMBER,\
                    LOT_NUMBER,\
                    TITLE,\
                    NUMBER_OFFERS_ELECTR,\
                    AWARD_EST_VALUE_EURO,\
                    AWARD_VALUE_EURO,\
                    VALUE_EURO_FIN_1_1,\
                    B_SUBCONTRACTED,\
                    B_EU_FUNDS,\
                    DT_AWARD\
                   
                   ]
    
    awardNoticeRow=[None if elt=="" else elt for elt in awardNoticeRow] 
    awardNoticeRow=tuple(awardNoticeRow)
    
    return awardNoticeRow

In [58]:
schema = StructType([
    StructField("Dispatch_Date", StringType(), False),
    StructField("Award_Notice_Id_Link", StringType(), False),
    StructField("Contracting_Authority_Country", StringType(), False),
    StructField("Contracting_Authority_Name", StringType(), False),
    StructField("Contractor_Country", StringType(), False),
    StructField("Contractor_Name", StringType(), False),
    StructField("Contract_Value_Euros", IntegerType(), False),
    StructField("Number_Offers_Received", IntegerType(), False),
    StructField("CPV_Code", StringType(), False),
    
    StructField("YEAR", IntegerType(), False),
    StructField("ID_TYPE", StringType(), False),
    StructField("XSD_VERSION", StringType(), False),
    StructField("CANCELLED", StringType(), False),
    
    StructField("CAE_NATIONALID", StringType(), False),
    StructField("CAE_ADDRESS", StringType(), False),
    StructField("CAE_TOWN", StringType(), False),
    StructField("CAE_POSTAL_CODE", StringType(), False),
    
    StructField("CAE_TYPE", StringType(), False),
    StructField("MAIN_ACTIVITY", StringType(), False),
    StructField("B_ON_BEHALF", StringType(), False),
    StructField("TYPE_OF_CONTRACT", StringType(), False),
    StructField("TAL_LOCATION_NUTS", StringType(), False),
    StructField("B_FRA_AGREEMENT", StringType(), False),
    StructField("B_DYN_PURCH_SYST", StringType(), False),
    StructField("ADDITIONAL_CPVS", StringType(), False),
    StructField("B_GPA", StringType(), False),
    StructField("VALUE_EURO_FIN_1", IntegerType(), False),
    StructField("VALUE_EURO_FIN_2", IntegerType(), False),
    StructField("TOP_TYPE", StringType(), False),
    StructField("CRIT_CODE", StringType(), False),
    StructField("CRIT_CRITERIA", StringType(), False),
    StructField("CRIT_WEIGHTS", StringType(), False),
    StructField("B_ELECTRONIC_AUCTION", StringType(), False),
    StructField("NUMBER_AWARDS", IntegerType(), False),
    
    StructField("WIN_ADDRESS", StringType(), False),
    StructField("WIN_TOWN", StringType(), False),
    StructField("WIN_POSTAL_CODE", StringType(), False),
    
    StructField("CONTRACT_NUMBER", StringType(), False),
    StructField("LOT_NUMBER", StringType(), False),
    StructField("TITLE", StringType(), False),
    StructField("NUMBER_OFFERS_ELECTR", IntegerType(), False),
    StructField("AWARD_EST_VALUE_EURO", IntegerType(), False),
    StructField("AWARD_VALUE_EURO", IntegerType(), False),
    StructField("VALUE_EURO_FIN_1_1", IntegerType(), False),
    StructField("B_SUBCONTRACTED", StringType(), False),
    StructField("B_EU_FUNDS", StringType(), False),
    StructField("DT_AWARD", StringType(), False)
    ])


## Load CSV data, convert, and save to parquet
CSV files are in ../data

Four files need to be processed:
* TED_CAN_2006.csv
* TED_CAN_2007.csv
* TED_CAN_2008.csv
* TED_CAN_2009_20015.csv

which can be downloaded from https://data.europa.eu/euodp/en/data/dataset/ted-csv


In [71]:
pathData='../data/TED_CAN_2009_2015.csv'
#pathData='../data/TED_CAN_2006.csv'


### Load

In [72]:
csvfile = sqlContext.read.format('com.databricks.spark.csv',).options(header='true', inferschema='true').load(pathData)#.repartition(2000)
csvfile.registerTempTable("csvData");


In [75]:
csvfile.take(1)

[Row(ID_NOTICE_CAN=u'201493', YEAR=u'2014', ID_TYPE=u'3', DT_DISPATCH=u'31-DEC-13', XSD_VERSION=u'R2.0.8.S02.E01', CANCELLED=u'0', CAE_NAME=u'Miejski Zarz\u0105d Dr\xf3g i Transportu w Cz\u0119stochowie', CAE_NATIONALID=u'', CAE_ADDRESS=u'ul. Popie\u0142uszki 4/6', CAE_TOWN=u'Cz\u0119stochowa', CAE_POSTAL_CODE=u'42-217', ISO_COUNTRY_CODE=u'PL', CAE_TYPE=u'3', MAIN_ACTIVITY=u'General public\\services', B_ON_BEHALF=u'N', TYPE_OF_CONTRACT=u'W', TAL_LOCATION_NUTS=u'', B_FRA_AGREEMENT=u'', B_DYN_PURCH_SYST=u'', CPV=u'45233141', ADDITIONAL_CPVS=u'45233142,45233222,45233223,45233140', B_GPA=u'N', VALUE_EURO=u'7684624.43', VALUE_EURO_FIN_1=u'7684624.42', VALUE_EURO_FIN_2=u'7684624.42', TOP_TYPE=u'OPE', CRIT_CODE=u'L', CRIT_CRITERIA=u'', CRIT_WEIGHTS=u'', B_ELECTRONIC_AUCTION=u'N', NUMBER_AWARDS=u'3', ID_AWARD=u'6074163', CONTRACT_NUMBER=u'923/2013', LOT_NUMBER=u'', TITLE=u'Bie\u017c\u0105ce utrzymanie dr\xf3g w granicach Miasta Cz\u0119stochowa w rejonie I w okresie od dnia 1.1.2014 r. do 31.1

### Process

In [73]:
processedData=csvfile.map(process)
df=processedData.toDF(schema)

### Save

In [74]:
df.write.mode('append').parquet('ted.parquet')

In [2]:
data=sqlContext.read.parquet("ted.parquet")

In [4]:
data.registerTempTable("csvData");

In [16]:
sqlContext.sql("select * from csvData limit 1").collect()

[Row(Dispatch_Date=u'2006-12-28', Award_Notice_Id_Link=u"<a href='http://ted.europa.eu/udl?uri=TED:NOTICE:60-2007:TEXT:EN:HTML' target='_blank'>60-2007</a>", Contracting_Authority_Country=u'France', Contracting_Authority_Name=u"Syndicat des Eaux d'Ile-de-France", Contractor_Country=u'France', Contractor_Name=u'Grpt EI TEM / Axeau', Contract_Value_Euros=3212853, Number_Offers_Received=3, CPV_Code=u'29566000', YEAR=2007, ID_TYPE=u'3', XSD_VERSION=u'D205          ', CANCELLED=u'0', CAE_NATIONALID=None, CAE_ADDRESS=u'14, rue Saint Beno\xeet', CAE_TOWN=u'Paris', CAE_POSTAL_CODE=u'75006', CAE_TYPE=u'3', MAIN_ACTIVITY=u'Environment', B_ON_BEHALF=u'N', TYPE_OF_CONTRACT=u'W', TAL_LOCATION_NUTS=None, B_FRA_AGREEMENT=None, B_DYN_PURCH_SYST=None, ADDITIONAL_CPVS=u'29122000,27220000,28814200,31200000', B_GPA=u'Y', VALUE_EURO_FIN_1=9751957, VALUE_EURO_FIN_2=9751957, CRIT_CODE=u'M', CRIT_CRITERIA=u'Prix.|Valeur technique.', CRIT_WEIGHTS=u'60|40', B_ELECTRONIC_AUCTION=u'N', NUMBER_AWARDS=4, WIN_ADDRES

In [40]:
sqlContext.sql("select distinct CRIT_CODE  from csvData").collect()

[Row(CRIT_CODE=u'L'), Row(CRIT_CODE=u'M'), Row(CRIT_CODE=u'')]

In [56]:
sqlContext.sql("select count(*)   from csvData where XSD_VERSION='D207          '").collect()

[Row(_c0=480127)]

In [13]:
sqlContext.sql("select count(*)   from csvData where Contracting_Authority_Country='Bulgaria' and YEAR='2006'").collect()

[Row(_c0=0)]